In [1]:
import tqdm.notebook as tqdm
import numpy as np
import scipy
import sklearn

# Spark

In [2]:
! /home/jovyan/start-hadoop.sh

jovyan
 * Starting OpenBSD Secure Shell server sshd
start-stop-daemon: unable to set gid to 0 (Operation not permitted)
   ...fail!
 * sshd is running
Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: namenode is running as process 160.  Stop it first and ensure /tmp/hadoop-jovyan-namenode.pid file is empty before retry.
Starting datanodes
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: datanode is running as process 270.  Stop it first and ensure /tmp/hadoop-jovyan-datanode.pid file is empty before retry.
Starting secondary namenodes [abdcfc9ca48e]
abdcfc9ca48e: Warning: Permanently added 'abdcfc9ca48e' (ED25519) to the list of known hosts.
abdcfc9ca48e: secondarynamenode is running as process 484.  Stop it first and ensure /tmp/hadoop-jovyan-secondarynamenode.pid file is empty before retry.
Starting resourcemanager
resourcemanager is running as process

In [3]:
# connect, context, session

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-04-05 16:55:20,923 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


## HDFS

In [4]:
! hdfs dfs -df -h

Filesystem               Size     Used  Available  Use%
hdfs://localhost:9000  58.4 G  291.6 M     29.8 G    0%


In [5]:
! hdfs dfs -ls /

Found 2 items
drwxrwx---   - root   supergroup          0 2023-04-05 16:55 /tmp
drwxr-xr-x   - jovyan supergroup          0 2023-04-05 16:55 /user


In [6]:
! mkdir -p ~/.kaggle

In [7]:
%%writefile ~/.kaggle/kaggle.json
{"username":"etagiev","key":"d6fca6c53559cee7d7996e4fb21b7543"}

Writing /home/jovyan/.kaggle/kaggle.json


In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
! pip install -U urllib3 kaggle==1.5.3
! kaggle competitions download -c outbrain-click-prediction -f page_views_sample.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f documents_topics.csv.zip 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.3 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.3-py3-none-any.whl size=64965 sha256=96219ef92dfa60c95451aa59d6fe6857bf9e3b2eee62db53c2b954263df33b62
  Stored in directory: /home/jovyan/.cache/pip/wheels/ec/9c/e3/864c634d8176e91b4845d2a12f7ae6b1c035d56457f81452d0
Successfully built kaggle
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15
 98%|███████████████████████████████████████▎| 146M/149M [00:05<00:00, 33.1MB/s]
100%|████████████████████████████████████████| 149M/149M [00:05<00:00, 26.9MB/s]
 99%|███████████████████████████████████████▋| 120M/121M [00:05<00:00, 30.0MB/s]
100%|

In [10]:
! unzip '*.zip'
! rm -rf *.zip

Archive:  page_views_sample.csv.zip
  inflating: page_views_sample.csv   

Archive:  documents_topics.csv.zip
  inflating: documents_topics.csv    

2 archives were successfully processed.


In [11]:
! hdfs dfs -put page_views_sample.csv
! hdfs dfs -put documents_topics.csv

In [12]:
# load data
page_views_sample = se.read.csv("page_views_sample.csv", header=True)
page_views_sample.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows



In [13]:
from IPython.display import display
tables = ["documents_topics", 
          "page_views_sample"]
for name in tqdm.tqdm(tables):
    df = se.read.csv("{}.csv".format(name), header=True)
    df.registerTempTable(name)
    print(name)
    df.limit(3).show()

  0%|          | 0/2 [00:00<?, ?it/s]

documents_topics
+-----------+--------+------------------+
|document_id|topic_id|  confidence_level|
+-----------+--------+------------------+
|    1595802|     140|0.0731131601068925|
|    1595802|      16|0.0594164867373976|
|    1595802|     143|0.0454207537554526|
+-----------+--------+------------------+

page_views_sample
+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
+--------------+-----------+---------+--------+------------+--------------+



In [14]:
page_views = se.table("page_views_sample")

In [15]:
documents_topics = se.table("documents_topics")

In [16]:
from pyspark.sql import functions as F
from pyspark.sql.functions import desc

## Evaluation Assignment

Data: outbrain click prediction

Tasks:
Using Spark RDD, DataFrame API and Python, calculate:

**1**. Top 10 most visited document_ids in the page_views_sample log

**2**. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)

**3***. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

The submission format is the result.json json file with top_10_documents, users and top_10_topics keys.
For TOP-10 results, the answer must be written in the form of a sheet ordered from TOP-1 to TOP-10 with an id.

result.json example:

    {
        "top_10_documents": [
            111,
            222,
            333,
            ...,
            1010
        ],
        "users": 10000,
        "top_10_topics": [
            11,
            22,
            33,
            ...,
            101
        ]
    }

## Task 1
Top 10 most visited document_ids in the page_views_sample log:

***If we group by 'document_id' column and get number of rows(count of visited document_ids), then with the sorting we can get top 10 most visited 'document_id's***:

In [28]:
 top_10_documents = (
    page_views.select(page_views.document_id)
      .groupBy('document_id')
      .count()
      .sort(F.desc('count'))
      .limit(10)
      ).rdd.map(lambda x: int(x.document_id)).collect()

In [29]:
 top_10_documents

[1811567,
 234,
 42744,
 1858440,
 1780813,
 60164,
 1790442,
 1877626,
 1821895,
 732651]

## Task 2
How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum):

***For this task, we can use 'countDistinct' function to get number of different traffic_sources for each user, or another version with the help of 'collect_set' which returns a set of objects with duplicate elements eliminated***:

In [19]:
#### version 1:

In [20]:
users = (page_views.groupby('uuid')
    .agg(F.countDistinct("traffic_source").alias("count_traffic_source"))
    .filter("count_traffic_source >= 2")
    .count())

In [21]:
users

98080

In [22]:
### version 2:

In [23]:
(page_views_sample.groupby('uuid')
    .agg(F.collect_set("traffic_source").alias("set_traffic_source"))
    .filter(F.size("set_traffic_source") >= 2)
    .count())

98080

## Task 3
Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

***Here we need additional step beyond "Task1", which is joining tables 'documents_topics' and 'page_views'. Then, grouping by 'topic_id' column and getting number of rows(count of visited topics), after that, by sorting we can get top 10 most visited topic_ids***:

In [30]:
top_10_topics = (
    page_views
      .join(documents_topics, page_views.document_id == documents_topics.document_id, 'inner')
      .select(documents_topics.topic_id)
      .groupBy('topic_id')
      .count()
      .sort(F.desc('count'))
      .limit(10)
).rdd.map(lambda x: int(x.topic_id)).collect()

In [31]:
top_10_topics

[20, 16, 216, 136, 140, 143, 36, 97, 8, 269]

In [58]:
result =  f'{{"top_10_documents": {top_10_documents}, "users": {users}, "top_10_topics": {top_10_topics} }}'

In [59]:
result

'{"top_10_documents": [1811567, 234, 42744, 1858440, 1780813, 60164, 1790442, 1877626, 1821895, 732651], "users": 98080, "top_10_topics": [20, 16, 216, 136, 140, 143, 36, 97, 8, 269] }'

In [62]:
with open('result.json', 'w') as outfile:
    outfile.write(result)

In [61]:
# with bash command:
! echo $result > result.json

In [63]:
! TG_USER=$afa_mammadli curl -F file=@result.json "51.250.54.133:80/MDS-LSML1/afa_mammadli/w4/1"

1.0
Well done!
